In [257]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from datetime import datetime

pd.set_option('display.max_rows', 5000)

In [258]:
zhi_df=pd.read_csv('../data/Zip_Zri_AllHomesPlusMultifamily.csv')


In [259]:
zhi_df2=pd.read_csv('../data/Zip_Zri_MultiFamilyResidenceRental.csv')


In [260]:
zhi_df.shape

(13181, 120)

In [261]:
zhi_df2.shape

(1861, 120)

In [262]:
date_mapping={i:j for i,j in enumerate(zhi_df.columns[6:])}

In [263]:
#zri = pd.melt(zhi_df, id_vars = zhi_df.columns[0:6], value_vars = zhi_df.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionID', 'year-month'])

zri = pd.melt(zhi_df, id_vars = zhi_df.columns[0:6], value_vars = zhi_df.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionID', 'year-month'])


zri=zri[(zri['year-month']>'2013-07') & (zri['year-month']<'2020-01')]

zri=zri.fillna(0)
zri['gapsize']=zri['ZRI'].rolling(5).sum()
droplist=list(zri[(zri['gapsize']==0) & (zri['year-month']>'2013-12') & (zri['year-month']<'2019-09')]['RegionID'].drop_duplicates())
zri[(zri['gapsize']==0)].shape[0]/zri.shape[0]
len(droplist)/len(zri['RegionID'].drop_duplicates())



0.16197557089750397

In [264]:
zri2 = pd.melt(zhi_df, id_vars = zhi_df.columns[0:6], value_vars = zhi_df.columns[7:], var_name = 'year-month', value_name = 'ZRI').sort_values(['RegionID', 'year-month'])

zri2=zri2.groupby('RegionID').apply(lambda group: group.interpolate())
#zri2[zri2['ZRI'].isna()]
zri2=zri2[(zri2['year-month']>'2013-12') & (zri2['year-month']<'2020-01')]

zri3=zri2[~(zri2['RegionID'].isin(droplist))]


In [265]:
def laggenerator(i,colname,df):
    timelist=list(df['year-month'].drop_duplicates().sort_values())[0:i]
    df.loc[:,f'{colname}_lag{i}']=df.loc[:,f'{colname}'].shift(i)
    df.loc[df['year-month'].isin(timelist),f'{colname}_lag{i}']=0
    return df

In [266]:
zri3=laggenerator(1, 'ZRI',zri3)
zri3=laggenerator(2, 'ZRI',zri3)
zri3=laggenerator(3, 'ZRI',zri3)

/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
/Users/moritz/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [267]:
train, test=zri3[(zri3['year-month']<='2018-12') & (zri3['year-month']>='2012-01')], zri3[(zri3['year-month']>'2018-12') & (zri3['year-month']<='2019-12')]

train=train.drop('year-month',axis=1)

Y_train, Y_test=train['ZRI'], test['ZRI']
X_train, X_test=train.drop('ZRI',axis=1), test.drop('ZRI',axis=1)



In [268]:
labelencoder = LabelEncoder()
X_train['CountyNameEnc'] = labelencoder.fit_transform(X_train['CountyName'])
X_train=X_train.drop(['CountyName','RegionID','RegionName','City','State','Metro'],axis=1)


In [269]:
cat=CatBoostRegressor()

In [270]:
cat.fit(X_train, Y_train)

Learning rate set to 0.124052
0:	learn: 511.8028566	total: 32.2ms	remaining: 32.2s
1:	learn: 454.2663010	total: 51ms	remaining: 25.4s
2:	learn: 402.9421765	total: 70.8ms	remaining: 23.5s
3:	learn: 358.5230865	total: 91.2ms	remaining: 22.7s
4:	learn: 319.6658776	total: 109ms	remaining: 21.7s
5:	learn: 285.6852176	total: 127ms	remaining: 21.1s
6:	learn: 255.9765179	total: 146ms	remaining: 20.7s
7:	learn: 229.8162657	total: 165ms	remaining: 20.5s
8:	learn: 206.8015427	total: 188ms	remaining: 20.7s
9:	learn: 186.7890120	total: 207ms	remaining: 20.5s
10:	learn: 169.9188227	total: 226ms	remaining: 20.3s
11:	learn: 154.9936932	total: 244ms	remaining: 20.1s
12:	learn: 142.3580027	total: 262ms	remaining: 19.9s
13:	learn: 131.2066367	total: 280ms	remaining: 19.7s
14:	learn: 122.0082058	total: 298ms	remaining: 19.6s
15:	learn: 114.1506279	total: 316ms	remaining: 19.4s
16:	learn: 107.5598632	total: 335ms	remaining: 19.4s
17:	learn: 102.0066307	total: 353ms	remaining: 19.3s
18:	learn: 97.4661492	to

161:	learn: 63.7978294	total: 3.4s	remaining: 17.6s
162:	learn: 63.6211038	total: 3.42s	remaining: 17.6s
163:	learn: 63.6121973	total: 3.45s	remaining: 17.6s
164:	learn: 63.6031199	total: 3.47s	remaining: 17.6s
165:	learn: 63.5852222	total: 3.5s	remaining: 17.6s
166:	learn: 63.5417598	total: 3.52s	remaining: 17.5s
167:	learn: 63.5339511	total: 3.54s	remaining: 17.6s
168:	learn: 63.5258104	total: 3.57s	remaining: 17.6s
169:	learn: 63.5168927	total: 3.6s	remaining: 17.6s
170:	learn: 63.4390131	total: 3.62s	remaining: 17.5s
171:	learn: 63.4320935	total: 3.64s	remaining: 17.5s
172:	learn: 63.4251447	total: 3.67s	remaining: 17.5s
173:	learn: 63.4167607	total: 3.69s	remaining: 17.5s
174:	learn: 63.3720463	total: 3.72s	remaining: 17.5s
175:	learn: 63.3173455	total: 3.74s	remaining: 17.5s
176:	learn: 63.2820038	total: 3.76s	remaining: 17.5s
177:	learn: 63.2636614	total: 3.78s	remaining: 17.5s
178:	learn: 63.1860852	total: 3.8s	remaining: 17.4s
179:	learn: 63.1787158	total: 3.83s	remaining: 17.

324:	learn: 59.7156151	total: 6.83s	remaining: 14.2s
325:	learn: 59.6958781	total: 6.85s	remaining: 14.2s
326:	learn: 59.6559016	total: 6.87s	remaining: 14.1s
327:	learn: 59.6528612	total: 6.89s	remaining: 14.1s
328:	learn: 59.6493481	total: 6.91s	remaining: 14.1s
329:	learn: 59.6458497	total: 6.93s	remaining: 14.1s
330:	learn: 59.6429249	total: 6.95s	remaining: 14s
331:	learn: 59.5783948	total: 6.97s	remaining: 14s
332:	learn: 59.5764936	total: 6.99s	remaining: 14s
333:	learn: 59.5553921	total: 7.01s	remaining: 14s
334:	learn: 59.5521179	total: 7.03s	remaining: 13.9s
335:	learn: 59.5494939	total: 7.05s	remaining: 13.9s
336:	learn: 59.5460294	total: 7.07s	remaining: 13.9s
337:	learn: 59.5404855	total: 7.08s	remaining: 13.9s
338:	learn: 59.4980491	total: 7.1s	remaining: 13.9s
339:	learn: 59.4938583	total: 7.13s	remaining: 13.8s
340:	learn: 59.4727891	total: 7.14s	remaining: 13.8s
341:	learn: 59.4287554	total: 7.16s	remaining: 13.8s
342:	learn: 59.3970275	total: 7.18s	remaining: 13.8s
34

486:	learn: 57.3311353	total: 10s	remaining: 10.6s
487:	learn: 57.3296658	total: 10.1s	remaining: 10.5s
488:	learn: 57.3271890	total: 10.1s	remaining: 10.5s
489:	learn: 57.3263122	total: 10.1s	remaining: 10.5s
490:	learn: 57.3162617	total: 10.1s	remaining: 10.5s
491:	learn: 57.2759035	total: 10.1s	remaining: 10.5s
492:	learn: 57.2468708	total: 10.2s	remaining: 10.4s
493:	learn: 57.2444319	total: 10.2s	remaining: 10.4s
494:	learn: 57.2022475	total: 10.2s	remaining: 10.4s
495:	learn: 57.2004492	total: 10.2s	remaining: 10.4s
496:	learn: 57.1984033	total: 10.2s	remaining: 10.4s
497:	learn: 57.1962496	total: 10.3s	remaining: 10.3s
498:	learn: 57.1941658	total: 10.3s	remaining: 10.3s
499:	learn: 57.1924785	total: 10.3s	remaining: 10.3s
500:	learn: 57.1907316	total: 10.3s	remaining: 10.3s
501:	learn: 57.1896874	total: 10.3s	remaining: 10.3s
502:	learn: 57.1787531	total: 10.4s	remaining: 10.2s
503:	learn: 57.1642954	total: 10.4s	remaining: 10.2s
504:	learn: 57.1634885	total: 10.4s	remaining: 1

642:	learn: 55.9311682	total: 13.2s	remaining: 7.32s
643:	learn: 55.9098722	total: 13.2s	remaining: 7.3s
644:	learn: 55.9088844	total: 13.2s	remaining: 7.28s
645:	learn: 55.9072724	total: 13.2s	remaining: 7.26s
646:	learn: 55.9053030	total: 13.3s	remaining: 7.24s
647:	learn: 55.8897012	total: 13.3s	remaining: 7.22s
648:	learn: 55.8882308	total: 13.3s	remaining: 7.2s
649:	learn: 55.8477749	total: 13.3s	remaining: 7.18s
650:	learn: 55.8463667	total: 13.4s	remaining: 7.16s
651:	learn: 55.8452314	total: 13.4s	remaining: 7.14s
652:	learn: 55.8436717	total: 13.4s	remaining: 7.13s
653:	learn: 55.8430670	total: 13.4s	remaining: 7.11s
654:	learn: 55.8423284	total: 13.5s	remaining: 7.09s
655:	learn: 55.8411906	total: 13.5s	remaining: 7.07s
656:	learn: 55.8401726	total: 13.5s	remaining: 7.05s
657:	learn: 55.8397544	total: 13.5s	remaining: 7.03s
658:	learn: 55.8390410	total: 13.5s	remaining: 7.01s
659:	learn: 55.8320691	total: 13.6s	remaining: 6.99s
660:	learn: 55.8311838	total: 13.6s	remaining: 6

799:	learn: 54.9378895	total: 16.6s	remaining: 4.14s
800:	learn: 54.9360562	total: 16.6s	remaining: 4.12s
801:	learn: 54.9356632	total: 16.6s	remaining: 4.1s
802:	learn: 54.9353569	total: 16.6s	remaining: 4.08s
803:	learn: 54.9294934	total: 16.6s	remaining: 4.05s
804:	learn: 54.9191438	total: 16.6s	remaining: 4.03s
805:	learn: 54.9185024	total: 16.7s	remaining: 4.01s
806:	learn: 54.9160765	total: 16.7s	remaining: 3.99s
807:	learn: 54.9124372	total: 16.7s	remaining: 3.97s
808:	learn: 54.9022333	total: 16.7s	remaining: 3.95s
809:	learn: 54.8958676	total: 16.7s	remaining: 3.93s
810:	learn: 54.8953556	total: 16.8s	remaining: 3.9s
811:	learn: 54.8949825	total: 16.8s	remaining: 3.88s
812:	learn: 54.8880595	total: 16.8s	remaining: 3.86s
813:	learn: 54.8875570	total: 16.8s	remaining: 3.84s
814:	learn: 54.8792861	total: 16.8s	remaining: 3.82s
815:	learn: 54.8765652	total: 16.9s	remaining: 3.8s
816:	learn: 54.8752372	total: 16.9s	remaining: 3.78s
817:	learn: 54.8704576	total: 16.9s	remaining: 3.

961:	learn: 54.2730739	total: 20s	remaining: 788ms
962:	learn: 54.2655559	total: 20s	remaining: 768ms
963:	learn: 54.2650132	total: 20s	remaining: 747ms
964:	learn: 54.2535604	total: 20s	remaining: 726ms
965:	learn: 54.2516839	total: 20s	remaining: 706ms
966:	learn: 54.2513536	total: 20.1s	remaining: 685ms
967:	learn: 54.2505853	total: 20.1s	remaining: 664ms
968:	learn: 54.2501228	total: 20.1s	remaining: 643ms
969:	learn: 54.2497424	total: 20.1s	remaining: 623ms
970:	learn: 54.2493637	total: 20.2s	remaining: 602ms
971:	learn: 54.2488681	total: 20.2s	remaining: 581ms
972:	learn: 54.2475713	total: 20.2s	remaining: 561ms
973:	learn: 54.2468986	total: 20.2s	remaining: 540ms
974:	learn: 54.2414522	total: 20.2s	remaining: 519ms
975:	learn: 54.2410124	total: 20.3s	remaining: 499ms
976:	learn: 54.2406165	total: 20.3s	remaining: 478ms
977:	learn: 54.2356101	total: 20.3s	remaining: 457ms
978:	learn: 54.2351526	total: 20.3s	remaining: 436ms
979:	learn: 54.2348687	total: 20.4s	remaining: 415ms
980

In [272]:
#loop months
#for January: filter Jan
#predict Jan
#update all lags
#loop

predictor_table=zri3.copy()

#loop through all month in 2019
for month in list(X_test['year-month'].drop_duplicates()):
    X_test2=X_test.copy()
    
    #run prediction for one month
    X_test2=X_test2[X_test2['year-month']==month]
    X_test2['CountyNameEnc'] = labelencoder.transform(X_test2['CountyName'])
    X_test2=X_test2.drop(['CountyName','RegionID','RegionName','City','State','Metro','year-month'],axis=1)
    val=cat.predict(X_test2)
    
    #define nextmonth
    nextmonth=date_mapping[[mon_ind for mon_ind, name in date_mapping.items() if name == month][0]+1]

    #write current month prediction into predictor_table
    if nextmonth<'2020-01':
        predictor_table.loc[predictor_table['year-month']==nextmonth,'ZRI']=val
        
        predictor_table=laggenerator(1, 'ZRI',predictor_table)
        predictor_table=laggenerator(2, 'ZRI',predictor_table)
        predictor_table=laggenerator(3, 'ZRI',predictor_table)
        
        X_test=predictor_table[predictor_table['year-month']>'2018-12']



In [273]:
RSME=(sum((Y_test-X_test['ZRI'])**2)/len(Y_test))**0.5
RSME

65.55435625218713

In [ ]:
#enc = OneHotEncoder(handle_unknown='ignore')

#X_train_enc = pd.DataFrame(enc.fit_transform(X_train[['CountyName']]).toarray())

#X_train = X_train.join(X_train_enc)
#X_train

#,

In [ ]:
testzip=zri3[zri3['RegionID']==753844]
testzip['year-month']=pd.to_datetime(testzip['year-month'])
testzip=testzip.set_index('year-month')
from statsmodels.tsa.seasonal import seasonal_decompose
import plotly.express as px

decomposed=seasonal_decompose(testzip['ZRI'],model='additive')

trend=decomposed.trend
seasonal=decomposed.seasonal
residual=decomposed.resid

#px.line(testzip['ZRI'])
#px.line(trend)
#px.line(seasonal)
#px.line(residual)

In [ ]:
from pmdarima.arima import auto_arima

x=100
X_train, X_test=testzip[0:x], testzip[x:len(testzip)-1]

arima_model=auto_arima(X_train['ZRI'], start_p=1, start_q=1, max_p=5, max_q=5, m=12, start_P=0, start_D=0,
                       start_Q=0, max_P=5, max_D=5, max_Q=5, 
                       seasonal=True, trace=True, stepwise=True, n_fits=50)

In [ ]:
zhi_reshaped2.gapsize==0

In [ ]:
zhi_reshaped2[(zhi_reshaped2['gapsize']==0)]

In [ ]:
zhi_reshaped2.fillna(0)
zhi_reshaped2['gapsize']=zhi_reshaped2['Value'].rolling(1).sum()

In [ ]:
zhi_reshaped2['gapsize']

In [ ]:
zhi_df.isna().sum(axis=1).shape

In [ ]:
zhi_df.isna().sum(axis=1)<1

In [ ]:
zhi_df[(zhi_df.isna().sum(axis=1)>8)].shape

In [ ]:
zhi_reshaped.groupby('RegionID').agg('count')

In [ ]:
zhi_reshaped

In [ ]:
acs_df=pd.read_csv('../data/acs_5yr_merged.csv')

In [ ]:
acs_df.shape

In [ ]:
acs_df.isna().sum()